In [1]:
import random as r
import json

from tensorflow import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from AbstractArchitecture_v2 import AbstractArchitecture
from DenseEncoder import DenseEncoder
from DenseDecoder import DenseDecoder
from NormalizedMeanSquaredError import NormalizedMeanSquaredError as NMSE
from plot_model_prediction import plot_model_prediction

In [2]:
# Set data file locations
data_file_prefix = './data/NLSL_expt1'

# Step 1. Load in the data
data_train_u = np.load("{}_train1_u.npy".format(data_file_prefix)).astype(np.float32)
data_train_f = np.load("{}_train1_f.npy".format(data_file_prefix)).astype(np.float32)
data_val_u = np.load("{}_val_u.npy".format(data_file_prefix)).astype(np.float32)
data_val_f = np.load("{}_val_f.npy".format(data_file_prefix)).astype(np.float32)
data_test_u1 = np.load("{}_test1_u.npy".format(data_file_prefix)).astype(np.float32)
data_test_f1 = np.load("{}_test1_f.npy".format(data_file_prefix)).astype(np.float32)
data_test_u = np.load("{}_test2_u.npy".format(data_file_prefix)).astype(np.float32)
data_test_f = np.load("{}_test2_f.npy".format(data_file_prefix)).astype(np.float32)


In [3]:
best_model_path = "./model_weights/NLSL_Experiment_06b/final_model"

# Load the best model
full_model = keras.models.load_model(best_model_path, 
                                     custom_objects={"NormalizedMeanSquaredError": NMSE})


In [4]:
loss_fns = 4*[NMSE()]
predicted_ys = full_model.predict(x=[data_train_u, data_train_f])

true_ys=[data_train_u, data_train_f, data_train_f, data_train_u]
nmse = NMSE()

total_loss = 0
for i, (pred_y, true_y, loss_fn) in enumerate(zip(predicted_ys, true_ys, loss_fns)):
    #print(pred_y.dtype, true_y.dtype)
    print("Score {}:".format(i+1), end="  ")
    loss = loss_fn(pred_y, true_y)
    tf.print(loss)
    total_loss += loss
    
print(data_train_u.shape)
print("Total Loss:", total_loss)

Score 1:  1.7438053e-05
Score 2:  4.44921898e-05
Score 3:  7.37037844e-05
Score 4:  0.0560517274
(8906, 128)
Total Loss: tf.Tensor(0.05618736, shape=(), dtype=float32)


In [5]:
val_x = [data_val_u, data_val_f]
val_ys = [data_val_u, data_val_f, data_val_f, data_val_u]
predicted_ys = full_model.predict(x=val_x)

total_loss = 0
for i, (pred_y, true_y, loss_fn) in enumerate(zip(predicted_ys, val_ys, loss_fns)):
    #print(pred_y.dtype, true_y.dtype)
    print("Score {}:".format(i), end="  ")
    loss = loss_fn(pred_y, true_y)
    tf.print(loss)
    total_loss += loss
    
print(data_train_u.shape)
print("Total Loss:", total_loss)

Score 0:  1.80504339e-05
Score 1:  0.000231853555
Score 2:  0.000150292646
Score 3:  0.0572284088
(8906, 128)
Total Loss: tf.Tensor(0.057628606, shape=(), dtype=float32)


In [6]:
full_model.evaluate(x=val_x, y=val_ys, batch_size=64)

35/35 [==============================] - 0s 3ms/step - loss: 0.0347 - output_1_loss: 1.8075e-05 - output_2_loss: 2.4335e-04 - output_3_loss: 1.4977e-04 - output_4_loss: 0.0340


[0.03474007919430733,
 1.8074726540362462e-05,
 0.00024334988847840577,
 0.0001497721386840567,
 0.03404095396399498]

In [22]:
i_hist = json.load(open('./model_weights/NLSL_Experiment_06b/model_history.json'))
m_hist = json.load(open('./model_weights/NLSL_Experiment_06b/initial_train.json'))
f_hist = json.load(open('./model_weights/NLSL_Experiment_06b/final_model_history.json'))

print(i_hist['loss'][-1], f_hist['loss'][0])
print(min(m_hist['full_init_loss']))

0.04213734716176987 0.03995571658015251
0.04019251341621081
